In [ ]:
# Instalação das bibliotecas necessárias
# Execute esta célula apenas uma vez
%pip install -q diffusers torch torchvision matplotlib pillow transformers accelerate


In [ ]:
# Importação das bibliotecas necessárias
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torchvision
from diffusers import DDPMScheduler, UNet2DModel, DDPMPipeline
from torchvision import transforms

# Configuração do dispositivo (GPU se disponível, senão CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# Função para visualizar imagens
def mostrar_imagens(imagens, titulo="Imagens"):
    """
    Função para mostrar uma grade de imagens de forma bonita
    """
    # Converte de (-1, 1) para (0, 1) para visualização
    imagens = imagens * 0.5 + 0.5
    
    # Cria uma grade de imagens
    grade = torchvision.utils.make_grid(imagens, nrow=4, padding=2)
    
    # Converte para formato PIL para visualização
    grade_im = grade.detach().cpu().permute(1, 2, 0).clip(0, 1)
    
    # Mostra a imagem
    plt.figure(figsize=(12, 8))
    plt.imshow(grade_im)
    plt.title(titulo)
    plt.axis('off')
    plt.show()

print("✅ Bibliotecas importadas com sucesso!")


In [ ]:
# Vamos criar algumas imagens de exemplo para demonstrar o processo
# Criaremos imagens simples com formas geométricas

def criar_imagem_exemplo(tamanho=64, tipo="circulo"):
    """
    Cria uma imagem de exemplo com formas simples
    """
    # Cria uma imagem em branco
    imagem = torch.zeros(3, tamanho, tamanho)
    
    if tipo == "circulo":
        # Cria um círculo no centro
        centro = tamanho // 2
        raio = tamanho // 4
        y, x = torch.meshgrid(torch.arange(tamanho), torch.arange(tamanho), indexing='ij')
        distancia = torch.sqrt((x - centro)**2 + (y - centro)**2)
        mascara = distancia <= raio
        imagem[:, mascara] = 1.0  # Círculo branco
        
    elif tipo == "quadrado":
        # Cria um quadrado no centro
        inicio = tamanho // 4
        fim = 3 * tamanho // 4
        imagem[:, inicio:fim, inicio:fim] = 1.0  # Quadrado branco
        
    elif tipo == "triangulo":
        # Cria um triângulo
        centro = tamanho // 2
        for i in range(tamanho // 4, 3 * tamanho // 4):
            largura = (i - tamanho // 4) * 2
            inicio = centro - largura // 2
            fim = centro + largura // 2
            if inicio >= 0 and fim < tamanho:
                imagem[:, i, inicio:fim] = 1.0
    
    return imagem

# Cria algumas imagens de exemplo
imagens_exemplo = []
tipos = ["circulo", "quadrado", "triangulo"]

for tipo in tipos:
    img = criar_imagem_exemplo(64, tipo)
    imagens_exemplo.append(img)

# Converte para tensor e mostra
imagens_tensor = torch.stack(imagens_exemplo)
mostrar_imagens(imagens_tensor, "Imagens de Exemplo - Formas Geométricas")


In [ ]:
# Vamos demonstrar o processo de adição de ruído
# Pegamos uma imagem limpa e adicionamos ruído gradualmente

def adicionar_ruido_simples(imagem, intensidade_ruido):
    """
    Adiciona ruído a uma imagem com intensidade controlada
    intensidade_ruido: 0 = sem ruído, 1 = ruído total
    """
    # Gera ruído aleatório
    ruido = torch.randn_like(imagem)
    
    # Mistura a imagem original com o ruído
    # Fórmula: imagem_ruidosa = (1 - intensidade) * imagem_original + intensidade * ruido
    imagem_ruidosa = (1 - intensidade_ruido) * imagem + intensidade_ruido * ruido
    
    return imagem_ruidosa

# Vamos pegar uma das nossas imagens de exemplo
imagem_original = imagens_exemplo[0]  # Círculo

# Criar diferentes níveis de ruído
intensidades = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
imagens_com_ruido = []

print("Demonstração do processo de adição de ruído:")
print("0.0 = Imagem original, 1.0 = Ruído puro")

for intensidade in intensidades:
    img_ruidosa = adicionar_ruido_simples(imagem_original, intensidade)
    imagens_com_ruido.append(img_ruidosa)

# Mostra todas as imagens em uma grade
imagens_ruido_tensor = torch.stack(imagens_com_ruido)
mostrar_imagens(imagens_ruido_tensor, "Processo de Adição de Ruído (0.0 → 1.0)")

# Mostra também os valores de intensidade
for i, intensidade in enumerate(intensidades):
    print(f"Imagem {i+1}: Intensidade = {intensidade}")


In [ ]:
# Criando um agendador DDPM (Denoising Diffusion Probabilistic Models)
# Este é um dos tipos mais comuns de agendadores

agendador = DDPMScheduler(
    num_train_timesteps=1000,  # Número de passos de ruído (mais passos = mais qualidade)
    beta_schedule="squaredcos_cap_v2"  # Tipo de cronograma de ruído
)

print("✅ Agendador criado com sucesso!")
print(f"Número de passos de treinamento: {agendador.num_train_timesteps}")

# Vamos visualizar como o ruído é adicionado ao longo do tempo
# O agendador tem parâmetros que controlam a intensidade do ruído em cada passo

# Plotando a curva de ruído
plt.figure(figsize=(10, 6))

# alphas_cumprod controla quanto da imagem original permanece
plt.plot(agendador.alphas_cumprod.cpu() ** 0.5, label="Sinal da imagem original", linewidth=2)
plt.plot((1 - agendador.alphas_cumprod.cpu()) ** 0.5, label="Intensidade do ruído", linewidth=2)

plt.xlabel("Passos de Ruído")
plt.ylabel("Intensidade")
plt.title("Como o Ruído é Adicionado Gradualmente")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("\n📊 Explicação do Gráfico:")
print("- Linha azul: Quanto da imagem original permanece")
print("- Linha laranja: Quanto de ruído foi adicionado")
print("- No início (passo 0): 100% imagem, 0% ruído")
print("- No final (passo 1000): 0% imagem, 100% ruído")


In [ ]:
# Agora vamos usar o agendador para adicionar ruído de forma mais sofisticada
# Vamos pegar uma de nossas imagens e adicionar ruído em diferentes passos

imagem_teste = imagens_exemplo[0].unsqueeze(0)  # Adiciona dimensão do batch
imagem_teste = imagem_teste.to(device)

# Vamos adicionar ruído em diferentes passos
passos_teste = [0, 200, 400, 600, 800, 999]  # Diferentes níveis de ruído
imagens_ruido_agendador = []

print("Demonstração usando o agendador DDPM:")
print("Passos diferentes = níveis diferentes de ruído")

for passo in passos_teste:
    # Cria ruído aleatório
    ruido = torch.randn_like(imagem_teste)
    
    # Cria tensor com o passo atual
    timesteps = torch.tensor([passo], device=device)
    
    # Usa o agendador para adicionar ruído
    imagem_ruidosa = agendador.add_noise(imagem_teste, ruido, timesteps)
    
    imagens_ruido_agendador.append(imagem_ruidosa.squeeze(0))

# Mostra as imagens
imagens_agendador_tensor = torch.stack(imagens_ruido_agendador)
mostrar_imagens(imagens_agendador_tensor, "Ruído Adicionado pelo Agendador DDPM")

# Mostra os passos
for i, passo in enumerate(passos_teste):
    print(f"Imagem {i+1}: Passo {passo} (ruído: {((1 - agendador.alphas_cumprod[passo]) ** 0.5):.3f})")


In [ ]:
# Criando um modelo UNet2D para nosso exemplo
# Vamos usar um modelo pequeno para demonstração

tamanho_imagem = 64  # Tamanho das imagens que vamos processar

modelo = UNet2DModel(
    sample_size=tamanho_imagem,        # Tamanho da imagem de entrada
    in_channels=3,                     # 3 canais (RGB)
    out_channels=3,                    # 3 canais de saída (RGB)
    layers_per_block=2,                # Camadas por bloco (mais = mais complexo)
    block_out_channels=(64, 64, 128, 128),  # Canais em cada bloco (mais = mais capacidade)
    down_block_types=(
        "DownBlock2D",                 # Blocos de redução (encolhem a imagem)
        "DownBlock2D", 
        "DownBlock2D",
        "DownBlock2D",
    ),
    up_block_types=(
        "UpBlock2D",                   # Blocos de expansão (aumentam a imagem)
        "UpBlock2D",
        "UpBlock2D", 
        "UpBlock2D"
    ),
)

# Move o modelo para o dispositivo (GPU se disponível)
modelo = modelo.to(device)

print("✅ Modelo UNet criado com sucesso!")
print(f"Parâmetros do modelo: {sum(p.numel() for p in modelo.parameters()):,}")

# Vamos testar se o modelo funciona corretamente
# Criamos uma imagem de teste com ruído
imagem_teste = torch.randn(1, 3, tamanho_imagem, tamanho_imagem).to(device)
timestep_teste = torch.tensor([500], device=device)  # Passo intermediário

# Testa o modelo
with torch.no_grad():
    predicao = modelo(imagem_teste, timestep_teste).sample

print(f"✅ Teste do modelo bem-sucedido!")
print(f"Entrada: {imagem_teste.shape}")
print(f"Saída: {predicao.shape}")
print("O modelo está pronto para ser treinado!")


In [ ]:
# Preparando os dados para treinamento
# Vamos usar nossas imagens de exemplo como dataset

# Converte nossas imagens para o formato correto
dataset_treino = torch.stack(imagens_exemplo).to(device)
print(f"Dataset de treino: {dataset_treino.shape}")

# Configuração do treinamento
otimizador = torch.optim.AdamW(modelo.parameters(), lr=1e-4)  # Taxa de aprendizado
num_epocas = 50  # Número de épocas (passadas completas pelos dados)
perdas = []  # Para acompanhar o progresso

print("🚀 Iniciando treinamento...")
print("Isso pode levar alguns minutos...")

# Loop de treinamento
for epoca in range(num_epocas):
    perda_epoca = 0
    num_batches = 0
    
    # Para cada imagem no nosso dataset
    for i in range(len(dataset_treino)):
        # Pega uma imagem limpa
        imagem_limpa = dataset_treino[i:i+1]  # Mantém dimensão do batch
        
        # Gera ruído aleatório
        ruido = torch.randn_like(imagem_limpa)
        
        # Escolhe um passo aleatório de ruído
        timesteps = torch.randint(
            0, agendador.num_train_timesteps, (1,), device=device
        ).long()
        
        # Adiciona ruído à imagem usando o agendador
        imagem_ruidosa = agendador.add_noise(imagem_limpa, ruido, timesteps)
        
        # Pergunta ao modelo: "Que ruído foi adicionado?"
        predicao_ruido = modelo(imagem_ruidosa, timesteps).sample
        
        # Calcula o erro (diferença entre ruído real e predição)
        perda = F.mse_loss(predicao_ruido, ruido)
        
        # Atualiza o modelo
        perda.backward()
        otimizador.step()
        otimizador.zero_grad()
        
        perda_epoca += perda.item()
        num_batches += 1
    
    # Calcula perda média da época
    perda_media = perda_epoca / num_batches
    perdas.append(perda_media)
    
    # Mostra progresso a cada 10 épocas
    if (epoca + 1) % 10 == 0:
        print(f"Época {epoca+1}/{num_epocas}, Perda: {perda_media:.6f}")

print("✅ Treinamento concluído!")


In [ ]:
# Vamos visualizar como o modelo aprendeu
# Plotando a curva de perda

plt.figure(figsize=(12, 5))

# Gráfico da perda
plt.subplot(1, 2, 1)
plt.plot(perdas, linewidth=2)
plt.title("Curva de Aprendizado")
plt.xlabel("Época")
plt.ylabel("Perda")
plt.grid(True, alpha=0.3)

# Gráfico da perda em escala logarítmica (para ver melhor a melhoria)
plt.subplot(1, 2, 2)
plt.plot(np.log(perdas), linewidth=2, color='orange')
plt.title("Curva de Aprendizado (Escala Log)")
plt.xlabel("Época")
plt.ylabel("Log da Perda")
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Análise da Curva de Aprendizado:")
print(f"Perda inicial: {perdas[0]:.6f}")
print(f"Perda final: {perdas[-1]:.6f}")
print(f"Melhoria: {((perdas[0] - perdas[-1]) / perdas[0] * 100):.1f}%")

if perdas[-1] < perdas[0] * 0.5:
    print("🎉 O modelo aprendeu bem! A perda diminuiu significativamente.")
else:
    print("⚠️ O modelo pode precisar de mais treinamento ou ajustes.")


In [ ]:
# Método 1: Usando o Pipeline (mais fácil)
# O pipeline combina o modelo e o agendador automaticamente

pipeline = DDPMPipeline(unet=modelo, scheduler=agendador)
pipeline = pipeline.to(device)

print("🎨 Gerando imagens com o pipeline...")

# Gera uma imagem
resultado = pipeline()
imagem_gerada = resultado.images[0]

# Mostra a imagem gerada
plt.figure(figsize=(8, 8))
plt.imshow(imagem_gerada)
plt.title("Imagem Gerada pelo Nosso Modelo!")
plt.axis('off')
plt.show()

print("🎉 Sucesso! Nossa primeira imagem gerada!")


In [ ]:
# Método 2: Loop de Geração Manual (para entender o processo)
# Vamos ver passo a passo como a imagem é gerada

print("🔍 Demonstração do processo de geração passo a passo...")

# Começa com ruído puro
amostra = torch.randn(1, 3, tamanho_imagem, tamanho_imagem).to(device)

# Lista para armazenar os passos intermediários
passos_intermediarios = []
passos_para_mostrar = [0, 200, 400, 600, 800, 999]  # Passos que queremos ver

# Loop de geração
for i, t in enumerate(agendador.timesteps):
    # Obtém a predição do modelo
    with torch.no_grad():
        predicao_ruido = modelo(amostra, t).sample
    
    # Atualiza a amostra removendo um pouco de ruído
    amostra = agendador.step(predicao_ruido, t, amostra).prev_sample
    
    # Salva alguns passos para visualização
    if i in passos_para_mostrar:
        passos_intermediarios.append(amostra.squeeze(0).cpu())

# Mostra o processo de geração
print("Processo de geração do ruído para imagem:")
imagens_processo = torch.stack(passos_intermediarios)
mostrar_imagens(imagens_processo, "Processo de Geração: Ruído → Imagem")

print("🎯 Explicação:")
print("- Imagem 1: Ruído puro (passo 0)")
print("- Imagem 2-5: Processo gradual de remoção de ruído")
print("- Imagem 6: Resultado final")


In [ ]:
# Vamos gerar várias imagens para ver a diversidade
print("🎲 Gerando múltiplas imagens para ver a diversidade...")

imagens_geradas = []
num_imagens = 8

for i in range(num_imagens):
    # Gera uma nova imagem
    resultado = pipeline()
    imagem = resultado.images[0]
    
    # Converte para tensor para visualização
    imagem_tensor = torch.tensor(np.array(imagem)).permute(2, 0, 1).float() / 255.0
    imagem_tensor = imagem_tensor * 2.0 - 1.0  # Normaliza para (-1, 1)
    imagens_geradas.append(imagem_tensor)

# Mostra todas as imagens geradas
imagens_tensor = torch.stack(imagens_geradas)
mostrar_imagens(imagens_tensor, "Diversidade de Imagens Geradas")

print("🎨 Cada imagem é única! O modelo gera variações diferentes a cada vez.")
print("Isso acontece porque começamos com ruído aleatório diferente a cada geração.")
